# **Preliminaries:** Install and import modules

In [1]:
#@title [RUN] install
!pip install networkx
!pip install mycolorpy
!pip install colorama
!pip install ogb

import torch
import os
!pip install torch-geometric torch-scatter torch-sparse torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for mycolorpy: filename=mycolorpy-1.5.1-py3-none-any.whl size=3874 sha256=e70cf7ca4a431bfdffb62d8bf0a26bc9b22abd749d3fe1670682dab80fcfefd8
  Stored in directory: /root/.cache/pip/wheels/b9/56/d6/a163bcbec3bb69f3f7797b1b542870b18d7e31ff5dbc0b87e3
Successfully built mycolorpy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 KB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=40714e7e3fff41bd1fec59e36a383ff0d356805deef0952bb0e6b2d40

In [21]:
#@title [RUN] Import modules
import numpy as np
import seaborn as sns
import math
import itertools
import scipy as sp
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric
from torch_geometric.datasets import Planetoid, Coauthor
from torch_scatter import scatter_mean, scatter_max, scatter_sum
from torch_geometric.utils import to_dense_adj
from torch.nn import Embedding
from torch_geometric.typing import Adj
from ogb.nodeproppred import PygNodePropPredDataset
from torch_geometric.loader import NeighborLoader
from torch_geometric.utils import to_scipy_sparse_matrix

#For FastRP
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix, spdiags
from sklearn.preprocessing import normalize, scale, MultiLabelBinarizer
from sklearn import random_projection


import pdb
from datetime import datetime

#for nice visualisations
import networkx as nx
import matplotlib.pyplot as plt

from mycolorpy import colorlist as mcp
import matplotlib.cm as cm

from typing import Mapping, Tuple, Sequence, List
import colorama

import scipy.linalg
from scipy.linalg import block_diag

In [3]:
####### PLOTS #######

def update_stats(training_stats, epoch_stats):
    """ Store metrics along the training
    Args:
      epoch_stats: dict containg metrics about one epoch
      training_stats: dict containing lists of metrics along training
    Returns:
      updated training_stats
    """
    if training_stats is None:
        training_stats = {}
        for key in epoch_stats.keys():
            training_stats[key] = []
    for key,val in epoch_stats.items():
        training_stats[key].append(val)
    return training_stats

def plot_stats(training_stats, figsize=(5, 5), name=""):
    """ Create one plot for each metric stored in training_stats
    """
    stats_names = [key[6:] for key in training_stats.keys() if key.startswith('train_')]
    f, ax = plt.subplots(len(stats_names), 1, figsize=figsize)
    if len(stats_names)==1:
        ax = np.array([ax])
    for key, axx in zip(stats_names, ax.reshape(-1,)):
        axx.plot(
            training_stats['epoch'],
            training_stats[f'train_{key}'],
            label=f"Training {key}")
        axx.plot(
            training_stats['epoch'],
            training_stats[f'val_{key}'],
            label=f"Validation {key}")
        axx.set_xlabel("Training epoch")
        axx.set_ylabel(key)
        axx.legend()
    plt.title(name)


def get_color_coded_str(i, color):
    return "\033[3{}m{}\033[0m".format(int(color), int(i))

def print_color_numpy(map, list_graphs):
    """ print matrix map in color according to list_graphs
    """
    list_blocks = []
    for i,graph in enumerate(list_graphs):
        block_i = (i+1)*np.ones((graph.num_nodes,graph.num_nodes))
        list_blocks += [block_i]
    block_color = block_diag(*list_blocks)
    
    map_modified = np.vectorize(get_color_coded_str)(map, block_color)
    print("\n".join([" ".join(["{}"]*map.shape[0])]*map.shape[1]).format(*[x for y in map_modified.tolist() for x in y]))

# Cora dataset



In [4]:
cora_dataset = Planetoid("/tmp/cora", name="cora", split="full")
cora_data = cora_dataset[0]
cora_data

Processing...
Done!


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [5]:
print("Training class sizes")
print(torch.bincount(cora_dataset[0].y[cora_dataset[0].train_mask]))
print("Validation class sizes")
print(torch.bincount(cora_dataset[0].y[cora_dataset[0].val_mask]))
print("Test class sizes")
print(torch.bincount(cora_dataset[0].y[cora_dataset[0].test_mask]))

Training class sizes
tensor([160,  90, 196, 341, 196, 138,  87])
Validation class sizes
tensor([ 61,  36,  78, 158,  81,  57,  29])
Test class sizes
tensor([130,  91, 144, 319, 149, 103,  64])


# OBGN-ARVIX dataset

In [6]:
d_name = "ogbn-arxiv"

dataset = PygNodePropPredDataset(name = d_name)

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
arxiv_data = dataset[0]
arxiv_data.y = arxiv_data.y.squeeze()
arxiv_data.node_year = arxiv_data.node_year.squeeze()
arxiv_data

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:07<00:00, 11.21it/s]


Extracting dataset/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 1792.44it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 4275.54it/s]

Saving...



Done!


Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343], y=[169343])

In [7]:
print("Training class sizes")
print(torch.bincount(arxiv_data.y[train_idx]))
print("Validation class sizes")
print(torch.bincount(arxiv_data.y[valid_idx]))
print("Test class sizes")
print(torch.bincount(arxiv_data.y[test_idx]))

Training class sizes
tensor([  437,   382,  3604,  1014,  2864,  2933,   703,   380,  4056,  2245,
         5182,   391,    21,  1290,   473,   248,  9998,   202,   402,  1873,
         1495,   304,  1268,  1539,  6989,   457,  2854,  1661, 16284,   239,
         4334,  1350,   270,   926,  5426,    75,  2506,  1615,  1100,  1551])
Validation class sizes
tensor([  74,  118,  502,  412, 1129,  779,  293,   75,  926,  230, 1232,  120,
           3,  440,   53,   68, 6846,  110,  138,  585,  268,   38,  249,  487,
        4458,  325,  710, 1074, 2273,   57, 2849,  586,   58,  125, 1027,   16,
         391,  273,  193,  209])
Test class sizes
tensor([   54,   187,   733,   654,  1869,  1246,   622,   134,  1250,   345,
         1455,   239,     5,   628,    71,    87, 10477,   203,   209,   419,
          313,    51,   386,   808, 10740,   475,  1041,  2066,  2849,   120,
         4631,   892,    83,   220,  1414,    36,   627,   481,   214,   269])


#Coauthor dataset

In [8]:
cs_dataset = Coauthor("/tmp/coauthor", name="CS")
cs_data = cs_dataset[0]
cs_data

Processing...
Done!


Data(x=[18333, 6805], edge_index=[2, 163788], y=[18333])

In [9]:
# Create manual split, do 60:20:20 across classes
num_classes_cs = 15
train_mask_cs_indices = []
val_mask_cs_indices = []
test_mask_cs_indices = []
cs_labels = cs_data.y
for i in range(num_classes_cs):

  class_i = np.where(cs_labels == i)[0]
  np.random.seed(0)
  np.random.shuffle(class_i)

  num_samples = len(class_i)
  train_mask_cs_indices += (class_i[:int(num_samples*0.6)]).tolist() 
  val_mask_cs_indices += (class_i[int(num_samples*0.6):int(num_samples*0.8)]).tolist() 
  test_mask_cs_indices += (class_i[int(num_samples*0.8):]).tolist() 

print(len(train_mask_cs_indices), len(val_mask_cs_indices), len(test_mask_cs_indices))
# Create the masks for training
# Test mask 
train_mask_cs = torch.full((len(cs_labels),), False)
train_mask_cs[train_mask_cs_indices] = True
# Val mask
val_mask_cs = torch.full((len(cs_labels),), False)
val_mask_cs[val_mask_cs_indices] = True
# Train mask
test_mask_cs = torch.full((len(cs_labels),), False)
test_mask_cs[test_mask_cs_indices] = True

10993 3668 3672


In [10]:
print("Training class sizes")
print(torch.bincount(cs_data.y[train_mask_cs]))
print("Validation class sizes")
print(torch.bincount(cs_data.y[val_mask_cs]))
print("Test class sizes")
print(torch.bincount(cs_data.y[test_mask_cs]))

Training class sizes
tensor([ 424,  277, 1230,  257,  836, 1315,  222,  554,  465,   70,  866, 1219,
         252, 2481,  525])
Validation class sizes
tensor([142,  92, 410,  86, 279, 439,  74, 185, 155,  24, 289, 407,  84, 827,
        175])
Test class sizes
tensor([142,  93, 410,  86, 279, 439,  75, 185, 155,  24, 289, 407,  84, 828,
        176])


# Data saving / loading

In [11]:
# use google drive for saving and loading information
from google.colab import drive
import pickle
import os

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/L45_project/'
# create folder if it does not exist already
if not os.path.exists(file_path):
  os.mkdir(file_path) 

Mounted at /content/drive


In [12]:
def save_training_info(training_stats: dict, node_embedding: torch.Tensor, filename: str):
  # write training data info to a file
  with open(file_path + filename + ".pkl", 'wb') as fp:
    pickle.dump(training_stats, fp)
    print('Training stats saved successfully to file: ' + filename)
  # write node embedding to a file
  torch.save(node_embedding, file_path + filename + "_emb.pt")
  print('Node embedding saved successfully to file: ' + filename)


def load_training_info(filename: str):
  # load training stats dictionary 
  with open(file_path + filename + ".pkl", 'rb') as fp:
    train_stats = pickle.load(fp)
    print('Training stats successfully loaded from file: ' + filename)
  # load node embedding
  node_embedding = torch.load(file_path + filename + "_emb.pt")
  print('Node embedding successfully loaded from file: ' + filename)
  return train_stats, node_embedding

# Final results is a list [seed, test result, [test per class accuracy], [training per class accuracy], [val per class accuracy]]
def save_final_results(final_results: List, filename: str):
  # write training data info to a file
  with open(file_path + filename + ".pkl", 'ab') as fp:
    pickle.dump(final_results, fp)
    print('Final results saved successfully to file: ' + filename)

# Returns an iterator which contains all the results from our various runs
def load_final_results(filename: str):
  with open(file_path + filename + ".pkl", 'rb') as fp:
    print('Final results found in file: ' + filename)
    while True:
      try:
        # This notation creates a generator, which we can then iterate through
        yield pickle.load(fp)
      except EOFError:
        break


In [ ]:
test_dict = {'c':[1,2,3], 'b':[4,5,6]}
test_tensor = torch.tensor([[1., -1.], [1., -1.]])
save_training_info(test_dict, test_tensor, "testing")
recovered_val1, recovered_val2 = load_training_info("testing")
print(recovered_val1, recovered_val2)

Training stats saved successfully to file: testing
Node embedding saved successfully to file: testing
Training stats successfully loaded from file: testing
Node embedding successfully loaded from file: testing
{'c': [1, 2, 3], 'b': [4, 5, 6]} tensor([[ 1., -1.],
        [ 1., -1.]])


# Model Wrappers

In [ ]:
from torch_geometric.nn import GCN

class GCNModelWrapper(GCN):

  def __init__(self, in_channels: int, hidden_channels: int, num_layers: int, out_channels: int):
    # use one less layer as our final graph layer can downsize for us
    # super().__init__(in_channels, hidden_channels, num_layers-1)
    super().__init__(in_channels, hidden_channels, num_layers)
    self.out_channels = out_channels
    self.final_layer = nn.Linear(hidden_channels, out_channels)

  def forward(self, x: torch.Tensor, edge_index: Adj):
    x = super().forward(x, edge_index)
    output = self.final_layer(x)
    return output


In [ ]:
from torch_geometric.nn import GAT

class GATModelWrapper(GAT):

  def __init__(self, in_channels: int, hidden_channels: int, num_layers: int, out_channels: int, v2: bool):
    # Create the model to extract the node embeddings then pass these through a linear layer for classification
    super().__init__(in_channels, hidden_channels, num_layers, v2=v2)
    self.out_channels = out_channels
    self.final_layer = nn.Linear(hidden_channels, out_channels)

  def forward(self, x: torch.Tensor, edge_index: Adj):
    x = super().forward(x, edge_index)
    output = self.final_layer(x)
    return output, x

In [ ]:
from torch_geometric.nn import GraphSAGE

class GraphSAGEModelWrapper(GraphSAGE):

  def __init__(self, in_channels: int, hidden_channels: int, num_layers: int, out_channels: int):
    # Create the model to extract the node embeddings then pass these through a linear layer for classification
    super().__init__(in_channels, hidden_channels, num_layers)
    self.out_channels = out_channels
    self.final_layer = nn.Linear(hidden_channels, out_channels)

  def forward(self, x: torch.Tensor, edge_index: Adj):
    x = super().forward(x, edge_index)
    output = self.final_layer(x)
    return output, x

In [ ]:
from torch_geometric.nn import Node2Vec
from torch import Tensor

class Node2VecWrapper(Node2Vec):
  def __init__(self, edge_index, embedding_size, walk_length, context_size, walks_per_node, num_negative_samples, p, q, sparse, out_channels):
    super().__init__(edge_index, embedding_dim=embedding_size, walk_length=walk_length,
                     context_size=context_size, walks_per_node=walks_per_node,
                     num_negative_samples=num_negative_samples, p=p, q=q, sparse=sparse)
    self.final_layer = nn.Linear(embedding_size, out_channels)
  def forward(self):
    x = super().forward()
    output = F.softmax(self.final_layer(x), dim=1)
    return output, x
  def test(
    self,
    train_z: Tensor,
    train_y: Tensor,
    test_z: Tensor,
    test_y: Tensor,
    solver: str = 'lbfgs',
    multi_class: str = 'auto',
    *args,
    **kwargs,
    ) -> float:
    r"""Evaluates latent space quality via a logistic regression downstream
    task."""
    from sklearn.linear_model import LogisticRegression

    clf = LogisticRegression(solver=solver, multi_class=multi_class, *args,
                            **kwargs).fit(train_z.detach().cpu().numpy(),
                                          train_y.detach().cpu().numpy())
    y_pred = clf.predict(test_z.detach().cpu().numpy())
    return y_pred

In [ ]:
from torch_geometric.nn import GIN

class GINWrapper(GIN):

  def __init__(self, in_channels: int, hidden_channels: int, num_layers: int, out_channels: int):
    # Create the model to extract the node embeddings then pass these through a linear layer for classification
    super().__init__(in_channels, hidden_channels, num_layers)
    self.out_channels = out_channels
    self.final_layer = nn.Linear(hidden_channels, out_channels)

  def forward(self, x: torch.Tensor, edge_index: Adj):
    x = super().forward(x, edge_index)
    output = self.final_layer(x)
    return output, x

# Training code



In [13]:
# @title [RUN] Hyperparameters GNN

NUM_EPOCHS_CORA =  10 #@param {type:"integer"}
NUM_EPOCHS_ARVIX =  110 #@param {type:"integer"}
LR         = 0.01 #@param {type:"number"}
HIDDEN_DIM = 128  #@param {type:"integer"}


In [ ]:
# Code taken from L45 practical notebook
def train_gnn(X, edge_indices, y, mask, model, optimiser, device):
    model.train()
    # Put data on device
    X = X.to(device)
    edge_indices = edge_indices.to(device)
    y = y.to(device)
    mask = mask.to(device)
    # Train
    optimiser.zero_grad()
    y_out, _ = model(X, edge_indices)
    y_hat = y_out[mask]
    loss = F.cross_entropy(y_hat, y)
    loss.backward()
    optimiser.step()
    return loss.data

# Training loop using subgraph batching from paper 'Inductive Representation Learning on Large Graphs' https://arxiv.org/pdf/1706.02216.pdf
def train_gnn_subgraph(data_batch, model, optimiser, device):
  total_loss = 0
  for batch in data_batch:
    # Put batch in device
    batch = batch.to(device)
    # Do training loop
    batch_size = batch.batch_size
    optimiser.zero_grad()
    y_out, _ = model(batch.x, batch.edge_index)
    y_out = y_out[:batch_size]
    batch_y = batch.y[:batch_size]
    batch_y = torch.reshape(batch_y, (-1,))
    loss = F.cross_entropy(y_out, batch_y)
    loss.backward()
    optimiser.step()
    # Keep a running total of the loss
    total_loss += float(loss)

  # Get the average loss across all the batches
  loss = total_loss / len(data_batch)
  return loss

def evaluate_gnn(X, edge_indices, y, mask, model, num_classes, device):
    model.eval()
    # Put data on device
    X = X.to(device)
    edge_indices = edge_indices.to(device)
    y = y.to(device)
    mask = mask.to(device)
    # Evaluate
    with torch.no_grad():
      y_out, node_embeddings = model(X, edge_indices)
    y_hat = y_out[mask]
    y_hat = y_hat.data.max(1)[1]
    num_correct = y_hat.eq(y.data).sum()
    num_total = len(y)
    accuracy = 100.0 * (num_correct/num_total)

    # calculate per class accuracy
    values, counts = torch.unique(y_hat[y_hat == y.data], return_counts=True)
    per_class_counts = torch.zeros(num_classes)
    # make sure per_class_counts is on the correct device
    per_class_counts = per_class_counts.to(device)
    # allocate the number of counts per class
    for i, x in enumerate(values):
      per_class_counts[x] = counts[i]
    # find total number of data points per class in the split
    total_per_class = torch.bincount(y.data)
    per_class_accuracy = torch.div(per_class_counts, total_per_class)

    return accuracy, per_class_accuracy, node_embeddings
    
# Training loop
def train_eval_loop_gnn(model, edge_indices, train_x, train_y, train_mask, valid_x, valid_y, valid_mask, 
                             test_x, test_y, test_mask, num_classes, seed, filename, device, Cora, subgraph_batches=None):
    optimiser = optim.Adam(model.parameters(), lr=LR)
    training_stats = None
    # Choose number of epochs
    NUM_EPOCHS = NUM_EPOCHS_CORA if Cora else NUM_EPOCHS_ARVIX
    # Training loop
    for epoch in range(NUM_EPOCHS):
        # If subgraph batching is not provided, use the full graph for training. Otherwise use subgraph batch training regime
        if subgraph_batches is None:
          train_loss = train_gnn(train_x, edge_indices, train_y, train_mask, model, optimiser, device)
        else:
          train_loss = train_gnn_subgraph(subgraph_batches, model, optimiser, device)
        # Calculate accuracy on full graph  
        train_acc, train_class_acc, _ = evaluate_gnn(train_x, edge_indices, train_y, train_mask, model, num_classes, device)
        valid_acc, valid_class_acc, _ = evaluate_gnn(valid_x, edge_indices, valid_y, valid_mask, model, num_classes, device)
        if epoch % 10 == 0 or epoch == (NUM_EPOCHS-1):
            print(f"Epoch {epoch} with train loss: {train_loss:.3f} train accuracy: {train_acc:.3f} validation accuracy: {valid_acc:.3f}")
            print("Per class train accuracy: ", train_class_acc)
            print("Per class val accuracy: ", valid_class_acc)
        # store the loss and the accuracy for the final plot
        epoch_stats = {'train_acc': train_acc, 'val_acc': valid_acc, 'epoch':epoch}
        training_stats = update_stats(training_stats, epoch_stats)

    # Lets look at our final test performance
    # Only need to get the node embeddings once, take from the training evaluation call
    test_acc, test_class_acc, node_embeddings = evaluate_gnn(test_x, edge_indices, test_y, test_mask, model, num_classes, device)
    print(f"Our final test accuracy for the GNN is: {test_acc:.3f}")
    print("Final per class accuracy on test set: ", test_class_acc)

    # Save training stats if on final iteration of the run
    save_training_info(training_stats, node_embeddings, filename+"_"+str(seed))
    # Save final results
    final_results_list = [seed, test_acc, test_class_acc, train_class_acc, valid_class_acc]
    save_final_results(final_results_list, filename)
    # Save final model weights incase we want to do further inference later
    torch.save(model.state_dict(), file_path+filename+"_" + str(seed) + "_model.pt")
    return training_stats

In [14]:
def set_seeds(seed):
  print("SETTING SEEDS TO: ", str(seed))
  # seed the potential sources of randomness
  torch.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)

In [25]:
# CHANGE: To name of model being tested
filename = "FastRP-arvix"
dataset = "Arvix"
# use 30 seeds which have been randomly generated using seed_list = [np.random.randint(4294967296 - 1) for i in range(30)]
seeds = [4193977854, 1863727779, 170173784, 2342954646, 116846604, 2105922959, 2739899259, 1024258131, 806299656, 880019963, 1818027900, 2135956485, 3710910636, 1517964140, 4083009686, 2455059856, 400225693, 89475662, 361232447, 3647665043, 1221215631, 2036056847, 1860537279, 516507873, 3692371949, 3300171104, 2794978777, 3303475786, 2952735006, 572297925]

# create folder for saving all model info into if it does not exist already
if not os.path.exists(file_path+filename+"/"):
  os.mkdir(file_path+filename+"/")

if dataset == "Cora":
  print("Using Cora dataset")
  # Get the edge indices and node features for our model. General set up variables for running with all the models
  edge_indices = cora_data.edge_index
  node_features = cora_data.x
  neighbour_dataset = cora_data

  # Get masks and training labels for each split
  train_mask = cora_data.train_mask
  train_y = cora_data.y[train_mask]
  valid_mask = cora_data.val_mask
  valid_y = cora_data.y[valid_mask]
  test_mask = cora_data.test_mask
  test_y = cora_data.y[test_mask]

  num_classes = 7
  is_cora=True

elif dataset=="Coauthor":
  print("Using Coauthor dataset")
  # Get the edge indices and node features for our model. General set up variables for running with all the models
  edge_indices = cs_data.edge_index
  node_features = cs_data.x
  neighbour_dataset = cs_data

  # Get masks and training labels for each split
  train_mask = train_mask_cs
  train_y = cs_data.y[train_mask]
  valid_mask = val_mask_cs
  valid_y = cs_data.y[valid_mask]
  test_mask = test_mask_cs
  test_y = cs_data.y[test_mask]

  num_classes = 15
  is_cora=True

# Otherwise we are using arvix dataset
else:
  print("Using Arvix dataset")
  # Get the edge indices and node features for our model
  edge_indices = arxiv_data.edge_index
  node_features = arxiv_data.x
  neighbour_dataset = arxiv_data

  # Get masks and training labels for each split
  train_mask = train_idx
  train_y = arxiv_data.y[train_mask]
  valid_mask = valid_idx
  valid_y = arxiv_data.y[valid_mask]
  test_mask = test_idx
  test_y = arxiv_data.y[test_mask]

  num_classes = 40
  is_cora = False


Using Arvix dataset


# Training Loops

In [ ]:
# Use to flush GPU memory if it gets too full
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# General training loop for all models except GraphSAGE, using the whole graph in training instead of using subgraph batching
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for seed in seeds:
  set_seeds(seed)
  # Create the model
  model = GATModelWrapper(in_channels = node_features.shape[-1], hidden_channels = HIDDEN_DIM, num_layers=1, out_channels=num_classes, v2=True)
  model = model.to(device)

  # Run training loop
  print("TRAINING WITH SEED: ", str(seed))
  train_stats_cora = train_eval_loop_gnn(model, edge_indices, node_features, train_y, train_mask, 
                                            node_features, valid_y, valid_mask, node_features, test_y, test_mask, num_classes, seed, filename+"/"+filename, device, is_cora)
  # Print out graphs if not using GPU
  if device == torch.device('cpu'):
    plot_stats(train_stats_cora, name=filename)

In [ ]:
# Training loop for GraphSAGE which using subgraph batches instead of the entire graph
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for seed in seeds:
  set_seeds(seed)
  # Original paper uses neighbourhood sizes  S1 = 25 and S2 = 10 so this is what we use
  train_loader = NeighborLoader(neighbour_dataset, num_neighbors = [25, 10], batch_size=1024, input_nodes=train_mask)

  # Create the model
  model = GraphSAGEModelWrapper(in_channels = node_features.shape[-1], hidden_channels = HIDDEN_DIM, num_layers=1, out_channels=num_classes)
  model = model.to(device)

  # Run training loop
  print("TRAINING WITH SEED: ", str(seed))
  train_stats_cora = train_eval_loop_gnn(model, edge_indices, node_features, train_y, train_mask, 
                                            node_features, valid_y, valid_mask, node_features, test_y, test_mask, num_classes, seed, filename+"/"+filename, device, is_cora, subgraph_batches=train_loader)
  # Print out graphs if not using GPU
  if device == torch.device('cpu'):
    plot_stats(train_stats_cora, name=filename)

In [ ]:
# Training loop for GraphSAGE which using subgraph batches instead of the entire graph
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for seed in seeds:
  set_seeds(seed)
  # Original paper uses neighbourhood sizes  S1 = 25 and S2 = 10 so this is what we use
  train_loader = NeighborLoader(neighbour_dataset, num_neighbors = [25, 10], batch_size=1024, input_nodes=train_mask)

  # Create the model
  model = GINWrapper(in_channels = node_features.shape[-1], hidden_channels = HIDDEN_DIM, num_layers=1, out_channels=num_classes)
  model = model.to(device)

  # Run training loop
  print("TRAINING WITH SEED: ", str(seed))
  train_stats_cora = train_eval_loop_gnn(model, edge_indices, node_features, train_y, train_mask, 
                                            node_features, valid_y, valid_mask, node_features, test_y, test_mask, num_classes, seed, filename+"/"+filename, device, is_cora, subgraph_batches=train_loader)
  # Print out graphs if not using GPU
  if device == torch.device('cpu'):
    plot_stats(train_stats_cora, name=filename)

# TESTING LOADING

In [ ]:
final_results = load_final_results(filename)
for r in final_results:
  print(r)

In [ ]:
training_stats_1, embedding = load_training_info(filename+"_1")
plot_stats(training_stats_1, name="Testing")
print(embedding)
print(node_features)

In [ ]:
# Loading stored model weights
model = GATModelWrapper(in_channels = node_features.shape[-1], hidden_channels = node_features.shape[-1], num_layers=1, out_channels=num_classes, v2=True)
model.load_state_dict(torch.load(file_path+filename+"/"+"GATV2_1_model.pt"))
model.eval()

- Plot graph with average training stats
- Save node embeddings for each run
- Save training stats for each run
- Save test accuracy for each run


# FastRP

In [17]:
class FastRPEmbeddingWrapper(nn.Module):
  def __init__(self, input_dim, num_classes):
      super().__init__()
      self.linear = nn.Linear(input_dim, num_classes)

  def forward(self, x):
      x = self.linear(x)
      return x

In [18]:
# Copied from https://github.com/GTmac/FastRP/blob/master/fastrp.py
# projection method: choose from Gaussian and Sparse
# input matrix: choose from adjacency and transition matrix
# alpha adjusts the weighting of nodes according to their degree
def fastrp_projection(A, q=3, dim=128, projection_method='gaussian', input_matrix='adj', alpha=None):
    assert input_matrix == 'adj' or input_matrix == 'trans'
    assert projection_method == 'gaussian' or projection_method == 'sparse'
    
    if input_matrix == 'adj':
        M = A
    else:
        N = A.shape[0]
        normalizer = spdiags(np.squeeze(1.0 / csc_matrix.sum(A, axis=1) ), 0, N, N)
        M = normalizer @ A
    # Gaussian projection matrix
    if projection_method == 'gaussian':
        transformer = random_projection.GaussianRandomProjection(n_components=dim, random_state=42)
    # Sparse projection matrix
    else:
        transformer = random_projection.SparseRandomProjection(n_components=dim, random_state=42)
    Y = transformer.fit(M)
    # Random projection for A
    if alpha is not None:
        Y.components_ = Y.components_ @ spdiags( \
                        np.squeeze(np.power(csc_matrix.sum(A, axis=1), alpha)), 0, N, N)
    cur_U = transformer.transform(M)
    U_list = [cur_U]
    
    for i in range(2, q + 1):
        cur_U = M @ cur_U
        U_list.append(cur_U)
    return U_list

# When weights is None, concatenate instead of linearly combines the embeddings from different powers of A
def fastrp_merge(U_list, weights, normalization=False):
    dense_U_list = [_U.todense() for _U in U_list] if type(U_list[0]) == csc_matrix else U_list
    _U_list = [normalize(_U, norm='l2', axis=1) for _U in dense_U_list] if normalization else dense_U_list

    if weights is None:
        return np.concatenate(_U_list, axis=1)
    U = np.zeros_like(_U_list[0])
    for cur_U, weight in zip(_U_list, weights):
        U += cur_U * weight
    # U = scale(U.todense())
    # U = normalize(U.todense(), norm='l2', axis=1)
    return scale(np.asarray(U.todense())) if type(U) == csr_matrix else scale(np.asarray(U))

# A is always the adjacency matrix
# the choice between adj matrix and trans matrix is decided in the conf
def fastrp_wrapper(A, conf):
    U_list = fastrp_projection(A,
                               q=len(conf['weights']),
                               dim=conf['dim'],
                               projection_method=conf['projection_method'],
                               input_matrix=conf['input_matrix'],
                               alpha=conf['alpha'],
    )
    U = fastrp_merge(U_list, conf['weights'], conf['normalization'])
    return U

In [19]:
# Code adpated from L45 practical notebook
def train_embedding_classifier(X, y, mask, model, optimiser, device):
    model.train()
    # Put data on device
    X = X.to(device)
    y = y.to(device)
    mask = mask.to(device)
    # Train
    optimiser.zero_grad()
    y_out = model(X)
    y_hat = y_out[mask]
    loss = F.cross_entropy(y_hat, y)
    loss.backward()
    optimiser.step()
    return loss.data

def evaluate_embedding_classifier(X, y, mask, model, num_classes, device):
    model.eval()
    # Put data on device
    X = X.to(device)
    y = y.to(device)
    mask = mask.to(device)
    # Evaluate
    with torch.no_grad():
      y_out = model(X)
    y_hat = y_out[mask]
    y_hat = y_hat.data.max(1)[1]
    num_correct = y_hat.eq(y.data).sum()
    num_total = len(y)
    accuracy = 100.0 * (num_correct/num_total)

    # calculate per class accuracy
    values, counts = torch.unique(y_hat[y_hat == y.data], return_counts=True)
    per_class_counts = torch.zeros(num_classes)
    # make sure per_class_counts is on the correct device
    per_class_counts = per_class_counts.to(device)
    # allocate the number of counts per class
    for i, x in enumerate(values):
      per_class_counts[x] = counts[i]
    # find total number of data points per class in the split
    total_per_class = torch.bincount(y.data)
    per_class_accuracy = torch.div(per_class_counts, total_per_class)

    return accuracy, per_class_accuracy
    
# Training loop
def train_eval_loop_embedding_classifier(model, embeddings, train_y, train_mask, 
                                         valid_y, valid_mask, test_y, test_mask, num_classes, seed, filename, device, Cora):
    optimiser = optim.Adam(model.parameters(), lr=LR)
    training_stats = None
    # Choose number of epochs
    NUM_EPOCHS = NUM_EPOCHS_CORA if Cora else NUM_EPOCHS_ARVIX
    # Training loop
    for epoch in range(NUM_EPOCHS):
        train_loss = train_embedding_classifier(embeddings, train_y, train_mask, model, optimiser, device)
        # Calculate accuracy on full graph  
        train_acc, train_class_acc = evaluate_embedding_classifier(embeddings, train_y, train_mask, model, num_classes, device)
        valid_acc, valid_class_acc = evaluate_embedding_classifier(embeddings, valid_y, valid_mask, model, num_classes, device)
        if epoch % 10 == 0 or epoch == (NUM_EPOCHS-1):
            print(f"Epoch {epoch} with train loss: {train_loss:.3f} train accuracy: {train_acc:.3f} validation accuracy: {valid_acc:.3f}")
            print("Per class train accuracy: ", train_class_acc)
            print("Per class val accuracy: ", valid_class_acc)
        # store the loss and the accuracy for the final plot
        epoch_stats = {'train_acc': train_acc, 'val_acc': valid_acc, 'epoch':epoch}
        training_stats = update_stats(training_stats, epoch_stats)

    # Lets look at our final test performance
    # Only need to get the node embeddings once, take from the training evaluation call
    test_acc, test_class_acc = evaluate_embedding_classifier(embeddings, test_y, test_mask, model, num_classes, device)
    print(f"Our final test accuracy for the GNN is: {test_acc:.3f}")
    print("Final per class accuracy on test set: ", test_class_acc)

    # Save training stats if on final iteration of the run, the node embeddings are actually passed in for training, where  
    node_embeddings = embeddings
    save_training_info(training_stats, node_embeddings, filename+"_"+str(seed))
    # Save final results
    final_results_list = [seed, test_acc, test_class_acc, train_class_acc, valid_class_acc]
    save_final_results(final_results_list, filename)
    # Save final model weights incase we want to do further inference later
    torch.save(model.state_dict(), file_path+filename+"_" + str(seed) + "_model.pt")
    return training_stats

In [26]:
# Use parameters from example in https://github.com/GTmac/FastRP/blob/master/fast-random-projection-blogcatalog.ipynb
# Except our input matrix in an adjacency matrix and since we are not tuning alpha we just set this to None
conf = {
        'projection_method': 'sparse',
        'input_matrix': 'adj',
        'weights': [0.0, 0.0, 1.0, 6.67],
        'normalization': True,
        'dim': HIDDEN_DIM,
        'alpha': None,
        'C': 0.1
    }

num_nodes = node_features.shape[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for seed in seeds:
  set_seeds(seed)
  # Convert adjacency matrix to scipy matrix
  adj_matrix = to_scipy_sparse_matrix(edge_indices)
  embeddings = fastrp_wrapper(adj_matrix, conf)
  # convert to tensor 
  embeddings = torch.from_numpy(embeddings)

  # Create the model
  model = FastRPEmbeddingWrapper(HIDDEN_DIM, num_classes)
  model = model.to(device)

  # Run training loop
  print("TRAINING WITH SEED: ", str(seed))
  train_stats_cora = train_eval_loop_embedding_classifier(model, embeddings, train_y, train_mask, 
                                             valid_y, valid_mask, test_y, test_mask, num_classes, seed, filename+"/"+filename, device, is_cora)
  # Print out graphs if not using GPU
  if device == torch.device('cpu'):
    plot_stats(train_stats_cora, name=filename)

SETTING SEEDS TO:  4193977854


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:240: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:259: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


TRAINING WITH SEED:  4193977854
Epoch 0 with train loss: 3.812 train accuracy: 12.637 validation accuracy: 11.054
Per class train accuracy:  tensor([0.0046, 0.0000, 0.0247, 0.0099, 0.0325, 0.0140, 0.0043, 0.0026, 0.0483,
        0.0196, 0.1416, 0.0128, 0.0000, 0.0047, 0.0042, 0.0000, 0.3104, 0.0248,
        0.0000, 0.0075, 0.0020, 0.0230, 0.0071, 0.0013, 0.0190, 0.0044, 0.1149,
        0.0054, 0.3362, 0.0126, 0.1147, 0.0148, 0.0000, 0.0032, 0.0713, 0.0000,
        0.0818, 0.0173, 0.0064, 0.0174], device='cuda:0')
Per class val accuracy:  tensor([0.0405, 0.0000, 0.0199, 0.0146, 0.0142, 0.0218, 0.0000, 0.0000, 0.0626,
        0.0261, 0.0471, 0.0167, 0.0000, 0.0023, 0.0000, 0.0000, 0.2832, 0.0636,
        0.0072, 0.0085, 0.0112, 0.0000, 0.0161, 0.0062, 0.0108, 0.0000, 0.1408,
        0.0326, 0.2635, 0.1053, 0.0832, 0.0068, 0.0000, 0.0000, 0.0662, 0.0000,
        0.1176, 0.0220, 0.0000, 0.0287], device='cuda:0')
Epoch 10 with train loss: 2.638 train accuracy: 34.548 validation accuracy: 39

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:240: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:259: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


TRAINING WITH SEED:  1863727779
Epoch 0 with train loss: 3.820 train accuracy: 9.595 validation accuracy: 10.393
Per class train accuracy:  tensor([0.0160, 0.0131, 0.0538, 0.0030, 0.0304, 0.0245, 0.0156, 0.0000, 0.6134,
        0.0334, 0.0417, 0.0486, 0.0000, 0.0023, 0.0063, 0.0000, 0.2377, 0.0495,
        0.0050, 0.0731, 0.0047, 0.0000, 0.0039, 0.0091, 0.0398, 0.0284, 0.1437,
        0.0132, 0.1011, 0.0042, 0.0353, 0.0326, 0.0000, 0.0270, 0.0442, 0.0000,
        0.0451, 0.0142, 0.0036, 0.0116], device='cuda:0')
Per class val accuracy:  tensor([0.0270, 0.0000, 0.0518, 0.0121, 0.0496, 0.0205, 0.0137, 0.0000, 0.3985,
        0.0304, 0.0211, 0.0167, 0.0000, 0.0045, 0.0000, 0.0000, 0.2894, 0.1727,
        0.0072, 0.0598, 0.0112, 0.0263, 0.0080, 0.0123, 0.0105, 0.0215, 0.1014,
        0.0047, 0.1144, 0.0175, 0.0193, 0.0102, 0.0000, 0.0240, 0.0516, 0.0000,
        0.0409, 0.0220, 0.0104, 0.0048], device='cuda:0')
Epoch 10 with train loss: 2.645 train accuracy: 32.222 validation accuracy: 39.

...

TRAINING WITH SEED:  3303475786
Epoch 0 with train loss: 3.842 train accuracy: 6.939 validation accuracy: 4.161
Per class train accuracy:  tensor([0.0069, 0.0052, 0.0408, 0.0069, 0.0468, 0.5742, 0.0128, 0.0000, 0.0182,
        0.1002, 0.0737, 0.0026, 0.0000, 0.0078, 0.0085, 0.0081, 0.0083, 0.0495,
        0.0373, 0.0128, 0.0154, 0.0000, 0.0016, 0.0279, 0.0240, 0.0438, 0.0662,
        0.0446, 0.1321, 0.0000, 0.0122, 0.0037, 0.0111, 0.0119, 0.0317, 0.0000,
        0.2175, 0.0031, 0.0036, 0.0168], device='cuda:0')
Per class val accuracy:  tensor([0.0000, 0.0000, 0.0239, 0.0073, 0.0567, 0.3132, 0.0375, 0.0000, 0.0292,
        0.1435, 0.0333, 0.0083, 0.0000, 0.0045, 0.0000, 0.2353, 0.0028, 0.2000,
        0.0580, 0.0222, 0.0075, 0.0000, 0.0201, 0.0267, 0.0182, 0.1108, 0.0549,
        0.1024, 0.1324, 0.0175, 0.0112, 0.0085, 0.0000, 0.0400, 0.0214, 0.0000,
        0.1714, 0.0037, 0.0000, 0.0191], device='cuda:0')
Epoch 10 with train loss: 2.661 train accuracy: 32.435 validation accuracy: 39.5

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:240: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:259: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


TRAINING WITH SEED:  2952735006
Epoch 0 with train loss: 3.908 train accuracy: 5.594 validation accuracy: 3.822
Per class train accuracy:  tensor([0.0572, 0.0026, 0.0241, 0.0148, 0.0070, 0.0419, 0.0071, 0.0000, 0.6208,
        0.0873, 0.0299, 0.0051, 0.0000, 0.0085, 0.0338, 0.0000, 0.0059, 0.0000,
        0.0000, 0.0064, 0.0100, 0.0033, 0.0055, 0.0208, 0.0288, 0.0547, 0.1671,
        0.1270, 0.0278, 0.0084, 0.0067, 0.0133, 0.0111, 0.0022, 0.0426, 0.0000,
        0.0375, 0.0080, 0.0055, 0.0142], device='cuda:0')
Per class val accuracy:  tensor([0.0000, 0.0000, 0.0239, 0.0194, 0.0230, 0.0116, 0.0205, 0.0133, 0.3758,
        0.1478, 0.0252, 0.0417, 0.0000, 0.0023, 0.0189, 0.0000, 0.0072, 0.0000,
        0.0072, 0.0171, 0.0261, 0.0000, 0.0000, 0.0246, 0.0119, 0.0769, 0.0972,
        0.1918, 0.0559, 0.0000, 0.0056, 0.0307, 0.0172, 0.0000, 0.0282, 0.0000,
        0.0537, 0.0293, 0.0104, 0.0144], device='cuda:0')
Epoch 10 with train loss: 2.666 train accuracy: 32.630 validation accuracy: 39.4

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:240: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_data.py:259: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


TRAINING WITH SEED:  572297925
Epoch 0 with train loss: 3.823 train accuracy: 7.453 validation accuracy: 5.564
Per class train accuracy:  tensor([0.0503, 0.0026, 0.0036, 0.0108, 0.0314, 0.5834, 0.0014, 0.0158, 0.0276,
        0.0205, 0.0826, 0.0026, 0.0000, 0.0054, 0.0000, 0.0081, 0.0014, 0.0050,
        0.0000, 0.0133, 0.0254, 0.0033, 0.0016, 0.0052, 0.0070, 0.0088, 0.1335,
        0.0096, 0.1589, 0.0084, 0.1059, 0.0800, 0.1111, 0.0097, 0.0131, 0.0000,
        0.1883, 0.0118, 0.0064, 0.0148], device='cuda:0')
Per class val accuracy:  tensor([0.0676, 0.0169, 0.0120, 0.0049, 0.0452, 0.3171, 0.0068, 0.0133, 0.0508,
        0.0087, 0.0438, 0.0083, 0.3333, 0.0091, 0.0000, 0.0000, 0.0032, 0.0182,
        0.0000, 0.0068, 0.0224, 0.0000, 0.0040, 0.0041, 0.0076, 0.0308, 0.0563,
        0.0093, 0.2248, 0.0175, 0.1330, 0.1877, 0.0690, 0.0080, 0.0117, 0.0000,
        0.1739, 0.0147, 0.0155, 0.0431], device='cuda:0')
Epoch 10 with train loss: 2.653 train accuracy: 34.609 validation accuracy: 39.32

# Node2Vec

In [ ]:
from torch_geometric.nn import Node2Vec
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

data_name = "Arxiv"

# Get masks and training labels for each split
if data_name == "Cora":
  num_classes = 7
  data = cora_data
  # Get the edge indices and node features for our model
  edge_indices = data.edge_index
  node_features = data.x
  # CHANGE: To name of model being tested
  filename =  "Node2Vec_Cora"
  train_mask = data.train_mask
  train_y = data.y[train_mask]
  valid_mask = data.val_mask
  valid_y = data.y[valid_mask]
  test_mask = data.test_mask
  test_y = data.y[test_mask]
elif data_name == "Coauthor":
  data = cs_data
  # Get the edge indices and node features for our model
  edge_indices = data.edge_index
  node_features = data.x
  num_classes = 15
  filename =  "Node2Vec_Coauthor_CS"
  train_mask = train_mask_cs
  train_y = data.y[train_mask]
  valid_mask = val_mask_cs
  valid_y = data.y[valid_mask]
  test_mask = test_mask_cs
  test_y = data.y[test_mask]
elif data_name == "Arxiv":
  data = arxiv_data
  edge_indices = arxiv_data.edge_index
  node_features = arxiv_data.x
  neighbour_dataset = arxiv_data

  # Get masks and training labels for each split
  train_mask = train_idx
  train_y = arxiv_data.y[train_mask]
  valid_mask = valid_idx
  valid_y = arxiv_data.y[valid_mask]
  test_mask = test_idx
  test_y = arxiv_data.y[test_mask]

  num_classes = 40
  is_cora = False

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# use 30 seeds which have been randomly generated using seed_list = [np.random.randint(4294967296 - 1) for i in range(30)]
seeds = [4193977854, 1863727779, 170173784, 2342954646, 116846604, 2105922959, 2739899259, 1024258131, 806299656, 880019963, 1818027900, 2135956485, 3710910636, 1517964140, 4083009686, 2455059856, 400225693, 89475662, 361232447, 3647665043, 1221215631, 2036056847, 1860537279, 516507873, 3692371949, 3300171104, 2794978777, 3303475786, 2952735006, 572297925]

# create folder for saving all model info into if it does not exist already
if not os.path.exists(file_path+filename+"/"):
  os.mkdir(file_path+filename+"/")

filename = filename + "/" + filename

for seed in seeds:
  set_seeds(seed)
  # Create the model
  #model = GATModelWrapper(in_channels = node_features.shape[-1], hidden_channels = node_features.shape[-1], num_layers=1, out_channels=num_classes, v2=True)
  model = Node2VecWrapper(data.edge_index.to(device), embedding_size=128, walk_length=20,
                     context_size=10, walks_per_node=10,
                     num_negative_samples=1, p=1, q=1, sparse=True, out_channels=num_classes).to(device)
  loader = model.loader(batch_size=128, shuffle=True,
                      num_workers=0)
  optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

  def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
      optimizer.zero_grad()
      loss = model.loss(pos_rw.to(device), neg_rw.to(device))
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
    return total_loss / len(loader)

  @torch.no_grad()
  def find_model_acc(model, train_z, train_y, test_z, test_y, solver: str = 'lbfgs', multi_class: str = 'auto', *args, **kwargs):
    pred_y = model.test(train_z, train_y, test_z, test_y, solver=solver, multi_class=multi_class, *args, **kwargs)
    acc = accuracy_score(test_y.detach().cpu().numpy(), pred_y)
    matrix = confusion_matrix(test_y.detach().cpu().numpy(), pred_y)
    per_class_acc = matrix.diagonal()/matrix.sum(axis=1)
    #print(m)
    #report = classification_report(test_y.detach().cpu().numpy(), pred_y)
    #print(report)
    return acc, per_class_acc

  @torch.no_grad()
  def test():
    model.eval()
    
    pred, z = model()
    acc_train, per_class_train_acc = find_model_acc(model, z[train_mask], data.y[train_mask],
                      z[train_mask], data.y[train_mask])
  
    acc_val, per_class_val_acc = find_model_acc(model, z[train_mask], data.y[train_mask],
                      z[valid_mask], data.y[valid_mask])

    acc_test, per_class_test_acc = find_model_acc(model, z[train_mask], data.y[train_mask],
                      z[test_mask], data.y[test_mask])

    return z, acc_train, per_class_train_acc, acc_val, per_class_val_acc, acc_test, per_class_test_acc

  training_stats = None
  for epoch in range(0, 10):
    loss = train()
    node_embeddings, acc_train, per_class_train_acc, acc_val, per_class_val_acc, acc_test, per_class_test_acc = test()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Acc_train: {acc_train:.4f}, Acc_val: {acc_val:.4f}, Acc_test: {acc_test:.4f}')
    print(f'Per class train accuracy: ', per_class_train_acc)
    epoch_stats = {'train_acc': acc_train, 'val_acc': acc_val, 'test_acc': acc_test, 'epoch':epoch}
    training_stats = update_stats(training_stats, epoch_stats)
  
  # Save training stats if on final iteration of the run
  save_training_info(training_stats, node_embeddings, filename+"_"+str(seed))
  # Save final results
  final_results_list = [seed, acc_test, per_class_test_acc, per_class_train_acc, per_class_val_acc]
  save_final_results(final_results_list, filename)
  # Save final model weights incase we want to do further inference later
  torch.save(model.state_dict(), file_path+filename+"_" + str(seed) + "_model.pt")

  plot_stats(training_stats, name=filename)

# Similarity tests

https://github.com/SGDE2020/embedding_stability/blob/master/similarity_tests/similarity_tests.py